<a href="https://colab.research.google.com/github/SridharSeshadri56/Inventory-Models/blob/main/SafetyStockModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Safety Stock Model

Enter demand (D) say per week, standard deviation of demand in same time units (sD), lead time to get supply (L) in same time units, and desired cycle service level (CSL) as a fraction CSL is the fraction of time have stock in a cycle. Typically between 0.85 and 0.99.

In [24]:
import numpy as np
from scipy import stats 
NormalDist = stats.norm

Here we enter the parameters of the model. 

In [25]:
#Check your inputs
D  = 2500 # Demand per period (for example per week)
sD = 500 # Standard deviation of demand (in the same unit as demand)
L  = 2 # weeks Replenishment lead time (this will be in same time unit)
CSL= 0.95 # Desired cycle service level 
timeunit = "week" #User enters this for printing pruposes only. Not used in calculations
Q = 10000  # Has to be entered by user if fill rate calculations are to carried out
# It is assumed that the order quantity has been derived previously.
# If fill rate is not desired, leave it blank.

In [26]:
print(colored("This is what you entered: \n________________________\n",color='blue'))
print("Demand                       = {} per {}".format(D, timeunit))
print("Standard deviation of demand = {} per {}".format(sD, timeunit))
print("Leadtime                     = {} {}s".format(L,timeunit))
print("Desired service level        = {}%".format(CSL*100.0))

This is what you entered: 
________________________

Demand                       = 2500 per week
Standard deviation of demand = 500 per week
Leadtime                     = 2 weeks
Desired service level        = 95.0%


Solution: Compute the reorder point (ROP) and safety stock ROP is the value of inventory position at which place an order Safety stock is the excess of stock over the mean demand during leadtime.

Intermediate Calculation
Mean demand during lead time: DL = D*L

SD of demand during lead time: sL = sD * sqrt(L)

In [27]:
DL = D*L
sL = sD*np.sqrt(L)
print(colored("Mean demand during leadtime  = {}".format(DL),color='blue'))
print("SD of demand during leadtime = {:.2f}".format(sL))

Mean demand during leadtime  = 5000
SD of demand during leadtime = 707.11


Safety stock is given by, ss = 𝑁𝑂𝑅𝑀𝑆𝐼𝑁𝑉(𝐶𝑆𝐿)×sL (here "S" stands for standard normal Reorder point ROP = DL + ss You can also directly compute ROP as NORMINV and directly enter mean and standard deviation along with CSL. In that case do not multiply by sL.

In [28]:
ss = NormalDist.isf(1-CSL, 0, 1)*sL
ROP = DL + ss
ROP1 = NormalDist.isf(1-CSL,DL, sL) # This gives same answer as above
print(colored(ROP == ROP1,color='blue'))
print("Safety Stock, ss   = {:.2f}".format(ss))
print("Reorder Point, ROP = {:.2f}".format(ROP))

True
Safety Stock, ss   = 1163.09
Reorder Point, ROP = 6163.09


Some what advanced material. Let us calculate fill rate. For this we need the order quantity (Q) 
𝐸𝑆𝐶= ss [1− NORMDIST(ss/𝜎_𝐿,0,1,1) + 𝜎_𝐿 𝑁𝑂𝑅𝑀𝐷𝐼𝑆𝑇(𝑠𝑠/𝜎_𝐿 , 0,1,0)).
fr = (Q - ESC)/Q. 
ESC stands for expected shortage per cycle. We give user chance to change Cycle Service Level if desired



In [32]:
print(f"Cycle service level was = {CSL*100}%")
inCSL = -1
while not(float(inCSL) and inCSL >0 and inCSL < 1):
    inCSL = input("Enter value of CSL between 0 and 1: ")
    try:
        inCSL = float(inCSL)
    except ValueError:
        print("CSL has to be between zero and 1, try again")
        inCSL = -1
    else:
        if not( inCSL >0 and inCSL < 1):
          print("You entered CSL = {}. Must be between 0 and 1. Try again! ".format(inCSL))

print(colored("You entered Cycle Service Level = {:.2f}%".format(inCSL*100), color = 'blue'))        
ss = NormalDist.isf(1-inCSL,DL, sL) - DL
ESC = -ss*(NormalDist.sf(ss/sL,0,1)) +sL*NormalDist.pdf(ss/sL,0,1)
print(colored("*********************************************\n",color='blue'))
print("Safety Stock, ss                     = {:.2f}".format(ss))
print("Reorder Point, ROP                   = {:.2f}".format(DL+ss))
print("The expected shortage per cycle, ESC = {:.2f}".format(ESC))
print("The fill rate, fr                    = {:.2f}%".format((Q-ESC)/Q*100))

Cycle service level was = 95.0%
Enter value of CSL between 0 and 1: 1.1
You entered CSL = 1.1. Must be between 0 and 1. Try again! 
Enter value of CSL between 0 and 1: .8
You entered Cycle Service Level = 80.00%
*********************************************

Safety Stock, ss                     = 595.12
Reorder Point, ROP                   = 5595.12
The expected shortage per cycle, ESC = 78.94
The fill rate, fr                    = 99.21%
